# Waterfilling Levels

In [ ]:
import sys
sys.path.insert(1, '../../functions')
import importlib
import numpy as np
import nbformat
import plotly.express
import plotly.express as px
import pandas as pd
import scipy.optimize as optimization
import food_bank_functions
import food_bank_bayesian
import matplotlib.pyplot as plt
import seaborn as sns
from food_bank_functions import *
from food_bank_bayesian import *
importlib.reload(food_bank_functions)

np.random.seed(1)

### Plotting the distribution

In [ ]:
vals = np.loadtxt('../../data/fbst_synthetic/vals.csv', delimiter=",")
support = np.loadtxt('../../data/fbst_synthetic/support.csv', delimiter=",")

In [ ]:
print(vals)
print(support)

In [ ]:
print(vals.shape)

In [ ]:
i=0
plt.plot(support[i], vals[i])

### Combining the Supports

In [ ]:
flat_support = support.flatten()
flat_vals = vals.flatten()
print(flat_support)
print(flat_vals)

In [ ]:
new_support, inverse_index = np.unique(flat_support, return_index=True)

In [ ]:
print(new_support)

In [ ]:
print(inverse_index)

In [ ]:
print(flat_support[84])

In [ ]:
print(new_support)

In [ ]:
new_weights = np.zeros((6, len(new_support)))
for i in range(new_weights.shape[0]):
    print('TOWN ' + str(i))
    print(support[i,:])
    print(vals[i,:])
    for j in range(new_weights.shape[1]):
        obs = new_support[j]
        if obs in support[i, :]:
            print('found value: ' + str(obs))
            new_index = np.argmin(np.abs(new_support - obs))
            old_index = np.argmin(np.abs(support[i,:] - obs))
            new_weights[i, new_index] += vals[i, old_index]

In [ ]:
for i in range(6):
    print(np.sum(new_weights[i,:]))

In [ ]:
i = 1
plt.plot(new_support, new_weights[i,:])

In [ ]:
weights = new_weights
print(weights)

In [ ]:
support = new_support
print(support)

In [ ]:
support.shape

In [ ]:
weights.shape

### Test

In [ ]:
n = 6
expected_demand = np.zeros(n)
for i in range(n):
    expected_demand[i] = np.dot(weights[i,:], support)
print(expected_demand)
max_budget = np.sum(expected_demand)

In [ ]:
print(max_budget)

In [ ]:
np.sum(vals)

In [ ]:
group_demands = np.zeros(n)
for i in range(n):
    group_demands[i] = np.random.choice(a=support, p=weights[i,:])
print(group_demands)

In [ ]:
print(waterfilling_waste(group_demands, max_budget))

In [ ]:
print(greedy(group_demands, max_budget))

In [ ]:
# print(constant_threshold(group_demands,max_budget,expected_demand))

In [ ]:
print(waterfilling_proportional_remaining(group_demands, max_budget))

In [ ]:
print(waterfilling_et_waste(expected_demand,group_demands,max_budget))

In [ ]:
print(waterfilling_et_full_waste(expected_demand,group_demands,max_budget))

In [ ]:
print(waterfilling_hope_waste(weights, support, group_demands, max_budget))

In [ ]:
print(waterfilling_hope_full_waste(weights, support, group_demands, max_budget))

In [ ]:
med = np.zeros(n)
var = np.zeros(n)
for i in range(n):
    med[i] = median(support, weights[i,:])
    var[i] = variance(support, weights[i,:])

In [ ]:
print(var)

In [ ]:
print(max_min_heuristic(group_demands, med, expected_demand, var, max_budget))

# Experiments

### Calculating Fairness Statistics


In [ ]:
expected_threshold = 0
for _ in range(1000):
    for i in range(n):
        group_demands[i] = np.random.choice(a=support, p=weights[i,:])
    opt = waterfilling_waste(group_demands,budget)
    expected_threshold += (1/1000)*max(opt)

In [ ]:
print(expected_threshold)

In [ ]:
print(support)

In [ ]:
num_iterations = 1000
n=6
sorted_distribution = support

In [ ]:
group = np.arange(n)
group_expected_demands = expected_demand
budget = np.sum(expected_demand)
score_hope_online = np.zeros((n,num_iterations))
score_hope_full = np.zeros((n,num_iterations))
score_et_online = np.zeros((n,num_iterations))
score_et_full = np.zeros((n,num_iterations))
score_greedy = np.zeros((n,num_iterations))
score_adapt_threshold = np.zeros((n,num_iterations))
score_fixed_threshold = np.zeros((n,num_iterations))
score_expect_threshold = np.zeros((n, num_iterations))
score_max_min = np.zeros((n, num_iterations))

# 8 different algorithms
env = np.zeros((9,num_iterations))
po = np.zeros((9,num_iterations))
prop = np.zeros((9,num_iterations))
linf = np.zeros((9,num_iterations))
l1 = np.zeros((9, num_iterations))
max_min = np.zeros((9, num_iterations))
for i in range(num_iterations):
    budget = np.sum(expected_demand)

        
    for j in range(n):
        group_demands[j] = np.random.choice(a=support, p=weights[j,:])
    group_median_demands = med
    group_variance = var
    

    
    opt = waterfilling_waste(group_demands,budget)


    
    hope_sol = waterfilling_hope_waste(weights, support, group_demands, budget)
    

    
    hope_full_sol = waterfilling_hope_full_waste(weights, support, group_demands, budget)
    dynamic = waterfilling_et_waste(group_expected_demands,group_demands,budget)
    et_full = waterfilling_et_full_waste(group_expected_demands, group_demands, budget)
    proportional_threshold = waterfilling_proportional_remaining(group_demands, budget)
    greedy_sol = greedy(group_demands,budget)
    threshold = constant_threshold(group_demands,budget, budget / n)
    expect_threshold = constant_threshold(group_demands, budget, expected_threshold)
    max_min_heuristic_sol = max_min_heuristic(group_demands, group_median_demands, group_expected_demands, group_variance, budget)

    # comparing hope_online
    
    score_hope_online[:,i] = opt - hope_sol
    env[0,i] = max(0,np.amax(envy_utility(hope_sol,group_demands)))
    po[0,i] = excess(hope_sol,budget)
    prop[0,i] = np.amax(proportionality_utility(hope_sol,group_demands,budget))
    max_min[0,i] = np.min(utility_ratio(hope_sol, group_demands, budget))
    linf[0,i] = np.amax(np.abs(score_hope_online[:,i]))
    l1[0,i] = np.sum(np.abs(score_hope_online[:,i]))
    
    
    # comparing hope_full
    
    score_hope_full[:,i] = opt - hope_full_sol
    env[1,i] = max(0,np.amax(envy_utility(hope_full_sol,group_demands)))
    po[1,i] = excess(hope_full_sol,budget)
    prop[1,i] = np.amax(proportionality_utility(hope_full_sol,group_demands,budget))
    max_min[1,i] = np.min(utility_ratio(hope_full_sol, group_demands, budget))
    linf[1,i] = np.amax(np.abs(score_hope_full[:,i]))
    l1[1, i] = np.sum(np.abs(score_hope_full[:,i]))
    
    
    # comparing et_online
    
    score_et_online[:,i] = opt - dynamic
    env[2,i] = max(0,np.amax(envy_utility(dynamic,group_demands)))
    po[2,i] = excess(dynamic,budget)
    prop[2,i] = np.amax(proportionality_utility(dynamic,group_demands,budget))
    max_min[2,i] = np.min(utility_ratio(dynamic, group_demands, budget))
    linf[2,i] = np.amax(np.abs(score_et_online[:,i]))
    l1[2,i] = np.sum(np.abs(score_et_online[:,i]))
    
    
    # comparing et_full
    
    score_et_full[:,i] = opt - et_full
    env[3,i] = max(0,np.amax(envy_utility(et_full,group_demands)))
    po[3,i] = excess(et_full,budget)
    prop[3,i] = np.amax(proportionality_utility(et_full,group_demands,budget))
    max_min[3,i] = np.min(utility_ratio(et_full, group_demands, budget))
    linf[3,i] = np.amax(np.abs(score_et_full[:,i]))
    l1[3,i] = np.sum(np.abs(score_et_full[:,i]))
    
    
    # comparing greedy
    
    score_greedy[:,i] = opt - greedy_sol
    env[4,i] = max(0,np.amax(envy_utility(greedy_sol,group_demands)))
    po[4,i] = excess(greedy_sol,budget)
    prop[4,i] = np.amax(proportionality_utility(greedy_sol,group_demands,budget))
    max_min[4,i] = np.min(utility_ratio(greedy_sol, group_demands, budget))
    linf[4,i] = np.amax(np.abs(score_greedy[:,i]))
    l1[4,i] = np.sum(np.abs(score_greedy[:,i]))
    
    # comparing fixed_threshold
    
    score_fixed_threshold[:,i] = opt - threshold
    env[5,i] = max(0,np.amax(envy_utility(threshold,group_demands)))
    po[5,i] = excess(threshold,budget)
    prop[5,i] = np.amax(proportionality_utility(threshold,group_demands,budget))
    max_min[5,i] = np.min(utility_ratio(threshold, group_demands, budget))
    linf[5,i] = np.amax(np.abs(score_fixed_threshold[:,i]))    
    l1[5,i] = np.sum(np.abs(score_fixed_threshold[:,i]))
    
    
    # comparing adaptive_threshold
    
    score_adapt_threshold[:,i] = opt - proportional_threshold
    env[6,i] = max(0,np.amax(envy_utility(proportional_threshold,group_demands)))
    po[6,i] = excess(proportional_threshold,budget)
    prop[6,i] = np.amax(proportionality_utility(proportional_threshold,group_demands,budget))
    max_min[6,i] = np.min(utility_ratio(proportional_threshold, group_demands, budget))
    linf[6,i] = np.amax(np.abs(score_adapt_threshold[:,i]))    
    l1[6,i] = np.sum(np.abs(score_adapt_threshold[:,i]))

    # comparing expected_threshold
    
    score_expect_threshold[:,i] = opt - expect_threshold
    env[7,i] = max(0,np.amax(envy_utility(expect_threshold,group_demands)))
    po[7,i] = excess(expect_threshold,budget)
    prop[7,i] = np.amax(proportionality_utility(expect_threshold,group_demands,budget))
    max_min[7,i] = np.min(utility_ratio(expect_threshold, group_demands, budget))
    linf[7,i] = np.amax(np.abs(score_expect_threshold[:,i]))   
    l1[7,i] = np.sum(np.abs(score_expect_threshold[:,i]))
    

    # comparing max_min_heurstic
    
    score_max_min[:,i] = opt - max_min_heuristic_sol
    env[8,i] = max(0,np.amax(envy_utility(max_min_heuristic_sol,group_demands)))
    po[8,i] = excess(max_min_heuristic_sol,budget)
    prop[8,i] = np.amax(proportionality_utility(max_min_heuristic_sol,group_demands,budget))
    max_min[8,i] = np.min(utility_ratio(max_min_heuristic_sol, group_demands, budget))
    linf[8,i] = np.amax(np.abs(score_max_min[:,i])) 
    l1[8,i] = np.sum(np.abs(score_max_min[:,i]))

In [ ]:
score_hope_online = np.average(score_hope_online, axis=1)
score_hope_full = np.average(score_hope_full, axis=1)
score_et_online = np.average(score_et_online, axis=1)
score_et_full = np.average(score_et_full, axis=1)
score_greedy = np.average(score_greedy, axis=1)
score_adapt_threshold = np.average(score_adapt_threshold, axis=1)
score_fixed_threshold = np.average(score_fixed_threshold, axis=1)
score_expect_threshold = np.average(score_expect_threshold, axis=1)
score_max_min = np.average(score_max_min, axis=1)

env_std = np.std(env, axis=1)
po_std = np.std(po, axis=1)
prop_std = np.std(prop, axis=1)
linf_std = np.std(linf, axis=1)
l1_std = np.std(l1, axis=1)
max_min_std = np.std(max_min, axis=1)


env = np.average(env,axis=1)
po = np.average(po,axis=1)
prop = np.average(prop,axis=1)
linf = np.average(linf,axis=1)
l1 = np.average(l1, axis=1)
max_min = np.average(max_min, axis=1)

In [ ]:
print(env)

In [ ]:
print(po)

In [ ]:
print(prop)

In [ ]:
print(max_min)

In [ ]:
print(linf)

In [ ]:
print('hope_online, hope_full, et_online, et_full, greedy, fixed_threshold, adaptive_threshold, expected_threshold, max_min')
print('envy:')
print(env)
print(env_std * 1.96 / np.sqrt(num_iterations))
print('po')
print(po)
print(po_std * 1.96 / np.sqrt(num_iterations))
print('prop')
print(prop)
print(prop_std * 1.96 / np.sqrt(num_iterations))
print('sum')
print(env+po+prop)
print('max_min')
print(max_min)
print(max_min_std * 1.96 / np.sqrt(num_iterations))
print('linf')
print(linf)
print(linf_std * 1.96 / np.sqrt(num_iterations))
print('l1')
print(l1)
print(l1_std * 1.96 / np.sqrt(num_iterations))

In [ ]:
data_dict = {'Group':group, 'hope_Online': score_hope_online, 'hope_Full':score_hope_full, 'et_Online':score_et_online, 'et_Full':score_et_full, 'Greedy':score_greedy, 'Adapt_Threshold': score_adapt_threshold, 'Fixed_Threshold': score_fixed_threshold, 'Expect_Threshold':score_expect_threshold, 'Max_Min_Heuristic':score_max_min}
df_uniform = pd.DataFrame(data_dict).melt(id_vars="Group")

In [ ]:
df_uniform.columns

In [ ]:
df_uniform.to_csv('fairness_group_by_group.csv')

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x='Group', y='value', hue = 'variable', data=df_uniform)
plt.title('Estimated Waterfilling Levels')
plt.xlabel('Estimated Level')